In [1]:
#path handling
import pathlib
import numpy as np

#out of memory computation
from dask_image.imread import imread
import tifffile

#set root dir
root = pathlib.Path(pathlib.Path.home(), 'home', 'Delta2_Caulobacter', 'Twitching')
im_path = '/Volumes/RG-UJ01-Data01/Andreas/2022-08_cdg_sensor_single_cell_analysis/RawData/Twitching/20220830_PAO1wt_Twitching0.5Agar0.3Tryp.tif'



#set number of color channels in image
n_channel = 3

im_stack = imread(im_path) #load image with dask-image for out of memory processing 
nframe = int(im_stack.shape[0]/n_channel)

newshape = (nframe, n_channel, *im_stack.shape[1:])
im_stack = im_stack.reshape(newshape)
    


ValueError: total size of new array must be unchanged

In [2]:
im_stack.shape

(1, 1024, 1024)

In [5]:

#set paths
root = pathlib.Path('/Volumes/RG-UJ01-Data01/Andreas/2022-08_cdg_sensor_single_cell_analysis')
data_dir = root / 'RawData' / 'Twitching' 

#create output dir
output_path = root / 'processed_dataTwitching'
(output_path).mkdir(exist_ok=True) #create output data folder,  each position will be placed in a subfolder

#get config file
config_file = root / 'config_2D_caulobacter.json'
cfg.load_config(config_file)
cfg.model_file_seg = to_str(root / 'models' / 'unet_pads_seg.hdf5') 
cfg.save_format = ('pickle','movie')

movie_names = [f.name for f in sorted((data_dir).glob('*.tif*'))]

print(movie_names)

Loading configuration from: /Volumes/RG-UJ01-Data01/Andreas/2022-08_cdg_sensor_single_cell_analysis/config_2D_caulobacter.json
['20220830_PAO1wt_Twitching0.5Agar0.3Tryp.tif']


In [6]:

for movie in movie_names:        
    #path to current position
    movie_dir = data_dir / movie
    
    #make nickname of movei (adapt to file name structure, here we take the part starting at TL and stopping before __R3D)
    start = movie.find('TL')
    end = movie.find('_R3D', start)
    movie_name_short = movie[start:end]
    
    #make subfolder for current position
    output_dir = output_path / movie_name_short
    (output_dir).mkdir(exist_ok=True)

    try:  
        print('starting with movie %s' %(movie_name_short)) 
        # Init reader (use bioformats=True if working with nd2, czi, ome-tiff etc):
        im_reader = xpreader(movie_dir, use_bioformats=True)

        # Print experiment parameters to make sure it initialized properly:
        print("""Initialized experiment reader:
            - %d positions
            - %d imaging channels
            - %d timepoints"""%(im_reader.positions, im_reader.channels, im_reader.timepoints)
        )

        # Init pipeline:
        xp = Pipeline(im_reader, resfolder=to_str(output_dir))   

        # Run pipeline
        xp.process()
        
    except:
        print('error with movie %s, skipping to next' %(movie_name_short)) 


#exit python
import os
os._exit(os.EX_OK)


starting with movie 
14:56:51.781 [Thread-1] DEBUG loci.common.NIOByteBufferProvider - Using mapped byte buffer? false
14:56:51.817 [Thread-1] DEBUG loci.formats.ClassList - Could not find loci.formats.in.URLReader
java.lang.ClassNotFoundException: loci.formats.in.URLReader
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:641) ~[na:na]
	at java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:188) ~[na:na]
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:521) ~[na:na]
	at java.base/java.lang.Class.forName0(Native Method) ~[na:na]
	at java.base/java.lang.Class.forName(Class.java:383) ~[na:na]
	at java.base/java.lang.Class.forName(Class.java:376) ~[na:na]
	at loci.formats.ClassList.parseLine(ClassList.java:196) ~[bioformats_package.jar:6.6.0]
	at loci.formats.ClassList.parseFile(ClassList.java:258) ~[bioformats_package.jar:6.6.0]
	at loci.formats.ClassList.<init>(ClassList.java:138) ~[bioformats_package.j

: 

: 